<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

## Generate code -> Refine code

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TV


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a comprehensive implementation of requesting an API using the `requests` library in Python. This example demonstrates how to make a GET request, handle the response, and manage potential errors.

```python
import requests

def fetch_data_from_api(url):
    """
    Fetch data from the provided API URL.

    Parameters:
    url (str): The API endpoint to fetch data from.

    Returns:
    dict: The JSON response from the API if the request was successful.
    None: If there was an error in the request.
    """
    try:
        # Make a GET request to the API
        response = requests.get(url)

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()
        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Log HTTP errors
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")  # Handle connection errors
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")  # Handle timeout errors
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")  # Handle all other requests issues

    return None

if __name__ == "__main__":
    # Example usage
    api_url = "https://api.example.com/data"  # Replace with your actual API URL
    result = fetch_data_from_api(api_url)

    if result:
        print("Data fetched successfully:")
        print(result)
    else:
        print("Failed to fetch data from the API.")
```

### Explanation:
- The `fetch_data_from_api` function takes a URL as its parameter and attempts to fetch data from the API.
- The `requests.get` function is used to send the GET request.
- It checks for HTTP errors using `raise_for_status()`, which raises an exception for any HTTP errors encountered.
- Various exceptions are caught and logged to provide feedback on what went wrong during the request.
- If the request is successful and the response is JSON, it is returned as a dictionary.
- The main block demonstrates how to use the function to retrieve data from the specified API URL. 

Make sure you have the `requests` library installed in your Python environment. You can install it via pip if you haven't done so already:

```bash
pip install requests
``` 

This should give you a solid foundation for making API requests using Python.

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your implementation of querying an API using the `requests` library is clear and serves its purpose well. Below are some critiques and recommendations to enhance your code quality, including error handling and overall best practices:

### Critiques and Recommendations

1. **Logging Mechanism**:
   - Currently, you're using print statements for error logging. It's recommended to use the logging module for more flexibility and configurability.
   ```python
   import logging

   logging.basicConfig(level=logging.ERROR)
   ```

   Replace the print statements with:
   ```python
   logging.error(f"HTTP error occurred: {http_err}")
   ```

2. **Function Return Type**:
   - Your function is expected to return `None` on failure and a dictionary on success. While this is common, it's a good practice to make this explicit in the function’s docstring by indicating the return type more clearly.

3. **Timeouts**:
   - Add a timeout parameter to your request to prevent hanging indefinitely if the API does not respond. A typical timeout can be around 5 seconds, but this can depend on your specific needs.
   ```python
   response = requests.get(url, timeout=5)
   ```

4. **Input Validation**:
   - You might want to check if the URL provided is valid before making the request. This could prevent unnecessary exceptions while attempting to connect to an invalid URL.
   ```python
   from urllib.parse import urlparse

   def validate_url(url):
       parsed = urlparse(url)
       return bool(parsed.scheme) and bool(parsed.netloc)
   ```

   In `fetch_data_from_api`, you can use this function to check the validity:
   ```python
   if not validate_url(url):
       logging.error("Invalid URL provided.")
       return None
   ```

5. **HTTP Method Support**:
   - `fetch_data_from_api` currently only supports GET requests. Consider making the function flexible to support other HTTP methods (e.g., POST, PUT, DELETE), which would make it more versatile.
   ```python
   def fetch_data_from_api(url, method='GET', data=None):
       # Use method variable to switch between request types
   ```

6. **Response Handling**:
   - You might want to consider handling cases where the API returns a non-200 status code but still provides useful information in the response body. This can be important for APIs that utilize non-standard response codes (like 429 for rate limiting).

7. **Improved User Feedback**:
   - When the main block does not receive data, it simply prints a failure message. It could be more informative by providing the error message as well, which would help in debugging.

8. **Type Annotations**:
   - You could enhance readability with type annotations in your function signature:
   ```python
   def fetch_data_from_api(url: str) -> dict:
   ```

### Revised Code Example
Here is an updated version considering some of the recommendations:

```python
import requests
import logging
from urllib.parse import urlparse

logging.basicConfig(level=logging.ERROR)

def validate_url(url: str) -> bool:
    parsed = urlparse(url)
    return bool(parsed.scheme) and bool(parsed.netloc)

def fetch_data_from_api(url: str) -> dict:
    """
    Fetch data from the provided API URL.

    Parameters:
    url (str): The API endpoint to fetch data from.

    Returns:
    dict: The JSON response from the API if the request was successful.
    None: If there was an error in the request.
    """
    if not validate_url(url):
        logging.error("Invalid URL provided.")
        return None

    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")

    return None

if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with your actual API URL
    result = fetch_data_from_api(api_url)

    if result:
        print("Data fetched successfully:")
        print(result)
    else:
        print("Failed to fetch data from the API.")
```

By implementing these suggestions, you will improve the robustness, flexibility, and maintainability of your API fetching code significantly.

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for your detailed critique and recommendations! Based on your feedback, I've revised the code to enhance its functionality, error handling, and overall best practices. Below is the updated implementation:

```python
import requests
import logging
from urllib.parse import urlparse

# Configure logging settings
logging.basicConfig(level=logging.ERROR)

def validate_url(url: str) -> bool:
    """
    Validate the provided URL.

    Parameters:
    url (str): The URL to validate.

    Returns:
    bool: True if the URL is valid, False otherwise.
    """
    parsed = urlparse(url)
    return bool(parsed.scheme) and bool(parsed.netloc)

def fetch_data_from_api(url: str, method: str = 'GET', data: dict = None) -> dict:
    """
    Fetch data from the provided API URL.

    Parameters:
    url (str): The API endpoint to fetch data from.
    method (str): The HTTP method to use (default is GET).
    data (dict): The data to send with the request (for POST, PUT).

    Returns:
    dict: The JSON response from the API if the request was successful.
    None: If there was an error in the request.
    """
    if not validate_url(url):
        logging.error("Invalid URL provided.")
        return None

    try:
        if method.upper() == 'POST':
            response = requests.post(url, json=data, timeout=5)
        else:  # Default to GET request
            response = requests.get(url, timeout=5)
        
        response.raise_for_status()  # Check for HTTP errors
        
        # Handle cases where the response is not a successful status
        return response.json() if response.status_code < 400 else None

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")

    return None

if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with your actual API URL
    result = fetch_data_from_api(api_url)

    if result:
        print("Data fetched successfully:")
        print(result)
    else:
        print("Failed to fetch data from the API, check the logs for details.")
```

### Improvements Made:
1. **Logging**: Replaced print statements with the `logging` module for better logging management.
2. **Function Signature**: Added a method parameter to support different HTTP methods, defaulting to GET.
3. **Input Validation**: Added URL validation before making requests to ensure proper URL format.
4. **Response Handling**: Enhanced the function to return `None` when HTTP status codes are not successful, while still providing error information via logging.
5. **Documentation**: Updated docstrings for clarity and added type hints for improved code readability.

These changes enhance the versatility, robustness, and maintainability of the code. Thank you again for your useful suggestions!

<br>

### We can of course make this a Class...
